# Imports

In [2]:
import pandas as pd
import numpy as np
import spreg
import libpysal
import geopandas as gpd
from libpysal.weights import Queen
from spreg import ML_Lag

# Data

In [3]:
# Read in the data
db = pd.read_csv('../_data/data_municipalities_final.csv')

# Train and test datasets (test on Wielkopolskie)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
db_train = db[db["municipality_code_str"].str[0:2] != "30"]
db_test = db[db["municipality_code_str"].str[0:2] == "30"]
# drop
db_train.drop(columns=["municipality_code_str"], inplace=True)
db_test.drop(columns=["municipality_code_str"], inplace=True)
print(db_train.shape, db_test.shape)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Train and test datasets
gdf_train = gdf[gdf["mncplty_c"].str[0:2] != "30"]
gdf_test = gdf[gdf["mncplty_c"].str[0:2] == "30"]
print(gdf_train.shape, gdf_test.shape)

/tmp/ipykernel_393029/4222980969.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_393029/4222980969.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)


(2251, 48) (226, 48)
(2251, 3) (226, 3)


# SAR Model

## On train dataset

In [6]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("municipality_name")
print(x_names)
x = db_train[x_names].values.astype('float64')
w_name = "border_contiguity"
w.transform = 'r'
mllag = ML_Lag(y=y,x=x,w=w,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True)

/tmp/ipykernel_393029/1104183743.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


['population_total', 'population_total_m', 'population_total_f', 'area_km2', 'population_density', 'urbanization_rate', 'healthcare_advices', 'population_per_pharmacy', 'installations_network_gas', 'appartments_per_1000_persons', 'persons_per_library', 'library_readers_per_1000_persons', 'forests_area', 'unemployment_rate', 'unemployment_rate_m', 'unemployment_rate_f', 'revenues_per_capita', 'revenues_per_capita_PIT', 'revenues_per_capita_CIT', 'net_scholarization', 'average_wage_relative', 'doctors_per_1000_persons', 'education_share_higher', 'education_share_secondary', 'education_share_vocational', 'education_share_primary', 'tourits_per_1000_persons', 'percent_population_below_25', 'percent_population_25_60', 'percent_population_over_60', 'election_presence', 'percent_invalid_votes', 'PIS_party', 'KO_party', 'KONFEDERACJA_party', 'PSL_party', 'SLD_party', 'partition_austrian', 'partition_prussian', 'partition_russian']


In [5]:
print(mllag.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          42
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2209
Pseudo R-squared    :      0.1256
Spatial Pseudo R-squared:  0.1370
Log likelihood      :  -6267.1964
Sigma-square ML     :     15.0680                Akaike info criterion :   12618.393
S.E of regression   :      3.8817                Schwarz criterion     :   12858.596

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------

### RMSE in-sample

In [60]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
mllag.predy
rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
print(rmse)

1.7487635476473689


## On test dataset

In [61]:
# firguring out betas and rho
# first beta is intercept
# last beta is rho
mllag.betas, mllag.rho

(array([[-3.46721754e+01],
        [-1.98181847e+07],
        [ 1.98181847e+07],
        [ 1.98181847e+07],
        [ 2.23403325e-03],
        [-5.36711039e-04],
        [ 9.12000787e-03],
        [-7.08041878e-07],
        [-2.20152232e-05],
        [-7.08957573e-03],
        [-3.11691898e-03],
        [-8.57309179e-06],
        [ 1.04768107e-03],
        [-3.17522809e-04],
        [ 1.89313899e+01],
        [-9.45738749e+00],
        [-9.09819112e+00],
        [ 5.95233636e-05],
        [-1.95080128e-04],
        [-4.80013029e-05],
        [-2.16734104e-03],
        [ 8.57773315e-03],
        [ 3.15582612e-03],
        [ 1.51778169e-01],
        [ 8.95963134e-02],
        [ 1.40411696e-01],
        [ 1.47515165e-01],
        [-3.67181526e-05],
        [ 2.46734645e-01],
        [ 2.50964296e-01],
        [ 2.84131509e-01],
        [ 1.80715019e-03],
        [-5.51296231e-02],
        [-5.82266870e-04],
        [ 4.42496436e-03],
        [-1.06635243e-02],
        [ 3.91338094e-03],
 

In [68]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
# Calculate y_pred
y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1]) + mllag.betas[-1]*Wy
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

15.682449430552015


/tmp/ipykernel_284493/895585023.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


# SDM

## In-sample

In [49]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("municipality_name")
x = db_train[x_names].values.astype('float64')
w_name = "border_contiguity"
w.transform = 'r'
mllag_SDM = ML_Lag(y,x,w,slx_lags=1,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_284493/2583859258.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


/home/maciej/anaconda3/envs/mlenv/lib/python3.8/site-packages/spreg/diagnostics.py:354: RuntimeWarning: invalid value encountered in sqrt
  se_result = np.sqrt(variance)
/home/maciej/anaconda3/envs/mlenv/lib/python3.8/site-packages/spreg/diagnostics.py:170: RuntimeWarning: invalid value encountered in sqrt
  ) / np.sqrt(variance)


In [50]:
print(mllag_SDM.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          82
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2169
Pseudo R-squared    :      0.1150
Spatial Pseudo R-squared:  0.0771
Log likelihood      :  -3919.8591
Sigma-square ML     :      1.7567                Akaike info criterion :    8003.718
S.E of regression   :      1.3254                Schwarz criterion     :    8472.687

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-St

### RMSE in-sample

In [51]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
mllag_SDM.predy
rmse = np.sqrt(np.sum((y - mllag_SDM.predy)**2)/len(y))
print(rmse)

1.3254092570371552


## On test dataset

In [52]:
# firguring out betas and rho
# first beta is intercept
# last beta is rho
mllag_SDM.betas, mllag_SDM.rho

(array([[ 9.93324983e+11],
        [-3.75002374e+08],
        [ 3.75002374e+08],
        [ 3.75002374e+08],
        [-4.75981110e-04],
        [-5.13270197e-04],
        [ 1.20542494e-02],
        [-7.44908774e-07],
        [ 1.15345361e-05],
        [ 2.93624490e-03],
        [-1.18803165e-03],
        [-1.13401327e-05],
        [-2.37143268e-04],
        [-6.64988237e-05],
        [-7.84724546e-01],
        [ 3.48031147e-01],
        [ 4.53727359e-01],
        [-7.21576400e-06],
        [-3.76353784e-04],
        [-6.89583354e-04],
        [-2.11252256e-03],
        [ 3.71367776e-02],
        [ 2.08902956e-02],
        [ 4.11748792e-01],
        [ 3.65064427e-01],
        [ 4.65694689e-01],
        [ 4.58634086e-01],
        [ 3.04258379e-05],
        [-1.14010212e+00],
        [-9.24367315e-01],
        [-1.33573513e+00],
        [ 6.76244774e-02],
        [ 9.85029761e-02],
        [ 1.63295069e-02],
        [ 3.56929774e-02],
        [-6.67668112e-02],
        [ 7.73648309e-03],
 

In [53]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
Wx = np.dot(w_array_normalized, db_test[x_names].values)
# Calculate y_pred
y_pred = mllag_SDM.betas[0] + np.dot(db_test[x_names].values,mllag_SDM.betas[1:-1][:40]) + mllag_SDM.betas[-1]*Wy + np.dot(Wx, mllag_SDM.betas[1:-1][40:])
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

/tmp/ipykernel_284493/2226874335.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


16.59794396195621
